In [1]:
!rm -rf /content/Moal
!git clone https://github.com/nqh-tq-32052503/Moal.git
%cd /content/Moal

Cloning into 'Moal'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (170/170), done.
remote: Compressing objects: 100% (137/137), done.
remote: Total 170 (delta 74), reused 63 (delta 30), pack-reused 0 (from 0)
Receiving objects: 100% (170/170), 460.22 KiB | 15.87 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/Moal


In [2]:
from Moal.backbone.vision_transformer_adapter import vit_base_patch16_224_bilora

/usr/local/lib/python3.12/dist-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/usr/local/lib/python3.12/dist-packages/timm/models/registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)


In [3]:
model = vit_base_patch16_224_bilora(pretrained=True)

/usr/local/lib/python3.12/dist-packages/timm/models/helpers.py:7: FutureWarning: Importing from timm.models.helpers is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [4]:
from Moal.utils.toolkit import count_parameters
count_parameters(model, trainable=True)

29071488

In [5]:
import torch

batch_size = 2 
dummy_input = torch.randn(
    batch_size,
    3, 224, 224,
    dtype=torch.float32,
    device="cpu"
)

print(dummy_input.shape)  # (B, 3, 224, 224)
print(dummy_input.dtype)  # torch.float32


torch.Size([2, 3, 224, 224])
torch.float32


In [6]:
y = model.forward(dummy_input, task=0)
y.shape

torch.Size([2, 768])

In [11]:
from Moal.backbone.vision_transformer_adapter import vit_base_patch16_224_adapter

In [12]:
from easydict import EasyDict
tuning_config = EasyDict(
    # AdaptFormer.
    ffn_adapt=True,
    ffn_option="parallel",
    ffn_adapter_layernorm_option="none",
    ffn_adapter_init_option="lora",
    ffn_adapter_scalar="0.1",
    ffn_num=64,
    d_model=768,
    # VPT related
    vpt_on=False,
    vpt_num=0,
)

In [13]:
ref_model = vit_base_patch16_224_adapter(num_classes=0,
                    global_pool=False, drop_path_rate=0.0, tuning_config=tuning_config)
ref_model.out_dim=768

I'm using ViT with adapters.


model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

_IncompatibleKeys(missing_keys=['blocks.0.adaptmlp.down_proj.weight', 'blocks.0.adaptmlp.down_proj.bias', 'blocks.0.adaptmlp.up_proj.weight', 'blocks.0.adaptmlp.up_proj.bias', 'blocks.1.adaptmlp.down_proj.weight', 'blocks.1.adaptmlp.down_proj.bias', 'blocks.1.adaptmlp.up_proj.weight', 'blocks.1.adaptmlp.up_proj.bias', 'blocks.2.adaptmlp.down_proj.weight', 'blocks.2.adaptmlp.down_proj.bias', 'blocks.2.adaptmlp.up_proj.weight', 'blocks.2.adaptmlp.up_proj.bias', 'blocks.3.adaptmlp.down_proj.weight', 'blocks.3.adaptmlp.down_proj.bias', 'blocks.3.adaptmlp.up_proj.weight', 'blocks.3.adaptmlp.up_proj.bias', 'blocks.4.adaptmlp.down_proj.weight', 'blocks.4.adaptmlp.down_proj.bias', 'blocks.4.adaptmlp.up_proj.weight', 'blocks.4.adaptmlp.up_proj.bias', 'blocks.5.adaptmlp.down_proj.weight', 'blocks.5.adaptmlp.down_proj.bias', 'blocks.5.adaptmlp.up_proj.weight', 'blocks.5.adaptmlp.up_proj.bias', 'blocks.6.adaptmlp.down_proj.weight', 'blocks.6.adaptmlp.down_proj.bias', 'blocks.6.adaptmlp.up_proj.wei

In [15]:
ref_y = ref_model.forward(dummy_input)
ref_y.shape

torch.Size([2, 768])